In [1]:
import cameo as cameo
from cobra import Model, Reaction, Metabolite
from collections import OrderedDict
import re
import escher
from cameo import load_model
# from cameo import Reaction, Metabolite
from cameo.data import metanetx
from cameo.network_analysis import model_to_network, remove_highly_connected_nodes
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.parallel import SequentialView
import os

In [2]:
from cameo.core import reaction, metabolite

In [3]:
model = load_model('iCL_Conversion1.json')

In [4]:
solution = model.solve()

In [5]:
# maps = escher.Builder(
#                     model=model,
#                     map_name = 'Ustilago maydis 521 genome-scale metabolic model',
#                     map_json='/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap3.json',
#                     reaction_data=solution.fluxes,
#                     show_gene_reaction_rules=True,
#                     use_3d_transform = True,
#                     )

# maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
#    { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
#    { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
# maps.display_in_browser()

In [6]:
import imp

In [7]:
foo = imp.load_source('curation_tools','/Users/clie/Desktop/EFPro2/Functions_Modules/curation_tools.py')

## Clean up NH4 Assimilation

In [8]:
for rxn in model.metabolites.nh4_c.reactions:
    if rxn.reversibility == True and model.metabolites.nh4_c in rxn.products:
        print rxn
        rxn.lower_bound = 0

_4__46__3__46__1__46__15__45__RXN: 23dappa_c + h2o_c + h_c <=> 2.0 nh4_c + pyr_c
_3__46__5__46__1__46__87__45__RXN: N__45__Carbamoyl__45__L__45__Amino__45__Acids_c + h2o_c + 2.0 h_c <=> L__45__2__45__AMINO__45__ACID_c + co2_c + nh4_c
RXN__45__1404: 2.0 h2o_c + ind3acnl_c <=> ind3ac_c + nh4_c
RXN__45__14728: ad_c + h2o_c <=> ac_c + nh4_c
AMP__45__DEAMINASE__45__RXN: amp_c + h2o_c + h_c <=> imp_c + nh4_c
RXN__45__14727: PROPIONAMIDE_c + h2o_c <=> nh4_c + ppa_c
RXN__45__9697: tyr__L_c <=> T4hcinnm_c + nh4_c
PHENYLALANINE__45__AMMONIA__45__LYASE__45__RXN: phe__L_c <=> cinnm_c + nh4_c
_2__46__3__46__2__46__13__45__RXN: Alkylamines_c + Protein__45__L__45__glutamine_c <=> Protein__45__N5__45__alkylglutamines_c + nh4_c
_3__46__5__46__5__46__1__45__RXN: Nitriles_c + 2.0 h2o_c <=> Carboxylates_c + nh4_c
RXN__45__14987: 34dhphe_c <=> 34dhcinm_c + nh4_c
RXNN__45__404: h2o_c + iad_c <=> ind3ac_c + nh4_c
BETA__45__UREIDOPROPIONASE__45__RXN: cala_c + h2o_c + 2.0 h_c <=> ala_B_c + co2_c + nh4_c
RXN__4

In [9]:
putative_nh4_assimilatory_rxns = []
for rxn in model.metabolites.nh4_c.reactions:
    if rxn.reversibility == True and model.metabolites.nh4_c in rxn.reactants:
        print rxn

GMP__45__SYN__45__NH3__45__RXN: atp_c + nh4_c + xmp_c <=> amp_c + gmp_c + 2.0 h_c + ppi_c
RXN__45__14325: atp_c + nh4_c + utp_c <=> adp_c + ctp_c + 2.0 h_c + pi_c
RXN__45__13854: 2.0 h2o_c + 3.0 nad_c + nh4_c <=> 5.0 h_c + 3.0 nadh_c + no2_c


In [10]:
# Removed because rxn GMPS_glu_m fulfills the same purpose 
# using Glutamate which is more correct than using Ammonia as the NH3-donor.
# This reaction would bypass the natural Ammonia Assimilation done via GS/GOGAT and NADP-GDH in
# U. maydis.

model.remove_reactions(['GMP__45__SYN__45__NH3__45__RXN'])

In [11]:
rxn = model.reactions.RXN__45__13854
print rxn
foo.invert_reaction(rxn)
rxn.lower_bound = 0
print rxn

RXN__45__13854: 2.0 h2o_c + 3.0 nad_c + nh4_c <=> 5.0 h_c + 3.0 nadh_c + no2_c
RXN__45__13854: 5.0 h_c + 3.0 nadh_c + no2_c --> 2.0 h2o_c + 3.0 nad_c + nh4_c


In [12]:
# Removed for the same reasons as GMP__45__SYN__45__NH3__45__RXN:
# Bypass of Ammonia Assimilation and alternative Reaction CTPS2 fulfills the same purpose
model.remove_reactions(['RXN__45__14325'])

In [13]:
putative_nh4_assimilatory_rxns = []
for rxn in model.metabolites.nh4_c.reactions:
    if rxn.reversibility == False and model.metabolites.nh4_c in rxn.reactants:
        print rxn

RXN__45__13202: 2.0 atp_c + hco3_c + nh4_c --> 2.0 adp_c + cbp_c + 2.0 h_c + pi_c
ASNSYNA__45__RXN: asp__L_c + atp_c + nh4_c --> amp_c + asn__L_c + h_c + ppi_c
GLUTDEHYD__45__RXN: akg_c + h_c + nadph_c + nh4_c --> glu__L_c + h2o_c + nadp_c
RXN__45__11338: CPD__45__12302_c + atp_c + nh4_c --> CPD__45__12295_c + amp_c + h_c + ppi_c
GLUTAMINESYN__45__RXN: atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c


In [14]:
# Removed for the same reasons as GMP__45__SYN__45__NH3__45__RXN:
# Bypass of Ammonia Assimilation and alternative Reaction ASNS1 fulfills the same purpose
model.remove_reactions(['ASNSYNA__45__RXN'])

In [15]:
# Removed for the same reasons as GMP__45__SYN__45__NH3__45__RXN:
# Bypass of Ammonia Assimilation and alternative Reaction HCGALm fulfills the same purpose
model.remove_reactions(['RXN__45__13202'])

In [16]:
# putative_nh4_assimilatory_rxns = []
# for rxn in model.metabolites.nh4_c.reactions:
#     if rxn.reversibility == False and model.metabolites.nh4_c in rxn.products:
#         print rxn

In [17]:
# GS/GOGAT -> correct IDs

In [18]:
model.reactions.GLUTAMINESYN__45__RXN.id = 'GLNS'
model.reactions.GDH1.id = 'GDHx'
model.reactions.GLUTDEHYD__45__RXN.id = 'GDHy'
model.reactions.TRANS__45__RXNTFS__45__43__45__AMMONIUM__47____47__AMMONIUM__46__19__46__.id = 'NH4t'

## Move Glyoxylate Shute to Mitochondrion

In [19]:
# The Glyoxylate Shunt is supposed to be located in the glyoxysome, since this compartment doesn't exist in the model
# MALSp and ISOCIT__45__CLEAV__45__RXN will be relocated to the mitochondrium

In [20]:
def move_compartment(model,reactions,target_compartment_string):
    for rxn in reactions:
        new_met_dict = {}
        for met in rxn.metabolites:
            query_id = met.id[:-2] + '_' + target_compartment_string
            if query_id in model.metabolites:
                new_met_dict[model.metabolites.get_by_id(query_id)] = rxn.metabolites[met]
                new_met_dict[met] = -rxn.metabolites[met]
            else:
                copy_met = met.copy()
                copy_met.id = query_id
                copy_met.compartment = target_compartment_string
                model.add_metabolite(copy_met)
                new_met_dict[model.metabolites.get_by_id(query_id)] = rxn.metabolites[met]
                new_met_dict[met] = -rxn.metabolites[met]
        rxn.add_metabolites(new_met_dict, combine=True)

In [21]:
move_compartment(model,[model.reactions.MALSp,model.reactions.ISOCIT__45__CLEAV__45__RXN],'m')

In [22]:
model.reactions.MALSp

Id,MALSp
Name,MALSYN-RXN
Stoichiometry,accoa_m + glx_m + h2o_m --> coa_m + h_m + mal__L_m
GPR,(um15004)
Lower bound,0.000000
Upper bound,25.000000


In [23]:
model.reactions.MALSp.id = 'MALSm'

In [24]:
rxn = model.reactions.MALATE__45__DEH__45__RXN_Cytosolic

## Fix Gene-Reaction-Rule for MDH and Malic Enzyme

In [25]:
rxn.id = 'MDH'

In [26]:
# um15004 was falsly annotated as a malate dehydrogenase
# more recently it was discovered that it functions as a malate synthase.
# um00403 is the mitochondrial malate dehydrogenase and um11161 is the cytosolic malate dehydrogenase
rxn.gene_reaction_rule = 'um11161'

In [27]:
rxn = model.reactions.MDHx
rxn.id = 'MDHm'
rxn.gene_reaction_rule = 'um00403'

In [28]:
model.reactions.Ubiquinol_Cytochrome_C_Reductase_RXN.gene_reaction_rule = '(um11097) and (um10507) and (um00940) and (um11212) and (um01478) and (um04237) and (um11288)'

In [29]:
model.genes.um11161.reactions

frozenset({<Reaction MALATE__45__DEHYDROGENASE__45__ACCEPTOR__45__RXN at 0x117578dd0>,
           <Reaction MDHy at 0x1177e2590>,
           <Reaction MDH at 0x120a4f5d0>,
           <Reaction ME1m at 0x12130e550>,
           <Reaction ME2m at 0x12130e690>})

In [30]:
# Malic enzyme um11919 catalyzes the reversible oxidative decarboxylation of malate to pyruvate and thus provides NADPH 
# for fatty acid biosynthesis
# Below I remove the NAD version of it and move the NADP reaction into the cytosol
model.remove_reactions(['ME1m','MALATE__45__DEHYDROGENASE__45__ACCEPTOR__45__RXN','MDHy'])

In [31]:
model.genes.um11161.reactions

frozenset({<Reaction MDH at 0x120a4f5d0>, <Reaction ME2m at 0x12130e690>})

In [32]:
model.reactions.ME2m.gene_reaction_rule = 'um11919'

In [33]:
move_compartment(model,[model.reactions.ME2m],'c')

In [34]:
model.reactions.ME2m.id = 'ME2'

## Glycolysis clean up

In [35]:
# MUMDB infers PFK to use ATP as its cofactor. Hence the PPI utilizing version of the reaction will be removed.

In [36]:
model.genes.um02512.reactions

frozenset({<Reaction PFKh at 0x12154f890>,
           <Reaction _2__46__7__46__1__46__90__45__RXN at 0x12154fc10>})

In [37]:
model.remove_reactions(['_2__46__7__46__1__46__90__45__RXN'])

In [38]:
model.genes.um15008.reactions

frozenset({<Reaction RXN__45__13720 at 0x1212d89d0>,
           <Reaction PGIBh at 0x12130d4d0>,
           <Reaction PGIAh at 0x1216bd810>})

In [39]:
# Generic phosphoglucose isomerase reaction removed
model.remove_reactions(['RXN__45__13720'])

In [40]:
# Glucose Metabolism Simplifictation. Right now the model has 3 isoforms metabolites for glucose (alpha,beta,both)
# This will be simplified to just use the one representing both.

In [41]:
import imp

In [42]:
foo = imp.load_source('curation_tools','/Users/clie/Desktop/EFPro2/Functions_Modules/curation_tools.py')

In [43]:
foo.find_reaction_from_metabolites(model,['glc__D_c','glc_A_c'])

--------------------------------------------------
These metabolites have no common reactions!


In [44]:
foo.swap_mets(model.metabolites.glc_A_c,model.metabolites.glc__D_c)

In [45]:
model.reactions.EX_glc_A_e.upper_bound = 100

In [46]:
model.reactions.EX_glc_A_e.lower_bound = -20

In [47]:
model.reactions.EX_glc_A_e.id = 'EX_glc__D_e'

In [48]:
model.add_metabolite(Metabolite('glc__D_e'))

In [49]:
foo.swap_mets(model.metabolites.glc_A_e,model.metabolites.glc__D_e)
model.metabolites.glc_A_e.remove_from_model('subtractive')

In [50]:
foo.swap_mets(model.metabolites.glc_bD_c,model.metabolites.glc__D_c)

In [51]:
model.metabolites.glc_bD_c.remove_from_model('subtractive')

In [52]:
model.metabolites.g6p_A_c.reactions

frozenset({<Reaction UG6PGTn at 0x12142b450>,
           <Reaction RXN__45__761 at 0x12143a750>,
           <Reaction PGIAh at 0x1216bd810>,
           <Reaction PGMTh at 0x1216bf050>,
           <Reaction G6PIh at 0x1216bf250>,
           <Reaction GLUKAh at 0x12189dd50>})

In [53]:
model.metabolites.g6p_B_c.reactions

frozenset({<Reaction PGIBh at 0x12130d4d0>,
           <Reaction G6PBDHh at 0x12130dcd0>,
           <Reaction G6PIh at 0x1216bf250>,
           <Reaction GLUCOKIN__45__RXN at 0x121889cd0>})

In [54]:
model.metabolites.D__45__glucopyranose__45__6__45__ehosphate_c.reactions

frozenset({<Reaction MYO__45__INOSITOL__45__1__45__PHOSPHATE__45__SYNTHASE__45__RXN at 0x117765510>,
           <Reaction HEX1 at 0x121889610>})

In [55]:
foo.swap_mets(model.metabolites.g6p_B_c,model.metabolites.g6p_A_c)

In [56]:
model.metabolites.g6p_A_c.reactions

frozenset({<Reaction PGIBh at 0x12130d4d0>,
           <Reaction G6PBDHh at 0x12130dcd0>,
           <Reaction UG6PGTn at 0x12142b450>,
           <Reaction RXN__45__761 at 0x12143a750>,
           <Reaction PGIAh at 0x1216bd810>,
           <Reaction PGMTh at 0x1216bf050>,
           <Reaction GLUCOKIN__45__RXN at 0x121889cd0>,
           <Reaction GLUKAh at 0x12189dd50>})

In [57]:
model.metabolites.g6p_B_c.reactions

frozenset()

In [58]:
foo.swap_mets(model.metabolites.D__45__glucopyranose__45__6__45__ehosphate_c,model.metabolites.g6p_A_c)

In [59]:
model.metabolites.g6p_A_c.reactions

frozenset({<Reaction MYO__45__INOSITOL__45__1__45__PHOSPHATE__45__SYNTHASE__45__RXN at 0x117765510>,
           <Reaction PGIBh at 0x12130d4d0>,
           <Reaction G6PBDHh at 0x12130dcd0>,
           <Reaction UG6PGTn at 0x12142b450>,
           <Reaction RXN__45__761 at 0x12143a750>,
           <Reaction PGIAh at 0x1216bd810>,
           <Reaction PGMTh at 0x1216bf050>,
           <Reaction HEX1 at 0x121889610>,
           <Reaction GLUCOKIN__45__RXN at 0x121889cd0>,
           <Reaction GLUKAh at 0x12189dd50>})

In [60]:
model.metabolites.g6p_A_c.name = 'D-Glucose 6-phosphate'

In [61]:
model.metabolites.g6p_A_c.id = 'g6p_c'

In [62]:
model.metabolites.g6p_c.reactions

frozenset({<Reaction MYO__45__INOSITOL__45__1__45__PHOSPHATE__45__SYNTHASE__45__RXN at 0x117765510>,
           <Reaction PGIBh at 0x12130d4d0>,
           <Reaction G6PBDHh at 0x12130dcd0>,
           <Reaction UG6PGTn at 0x12142b450>,
           <Reaction RXN__45__761 at 0x12143a750>,
           <Reaction PGIAh at 0x1216bd810>,
           <Reaction PGMTh at 0x1216bf050>,
           <Reaction HEX1 at 0x121889610>,
           <Reaction GLUCOKIN__45__RXN at 0x121889cd0>,
           <Reaction GLUKAh at 0x12189dd50>})

In [63]:
model.reactions.PGIBh.id = 'PGI'
model.reactions.PGMTh.id = 'PGMT'
model.reactions.G6PBDHh.id = 'G6PBDH'
model.reactions.UG6PGTn.id = 'UG6PGT'
model.reactions.MYO__45__INOSITOL__45__1__45__PHOSPHATE__45__SYNTHASE__45__RXN.id = 'MI1PS'

In [64]:
model.remove_reactions(['GLUKAh','PGIAh','GLUCOKIN__45__RXN'])

In [65]:
model.metabolites.f6p_c.reactions

frozenset({<Reaction RXN__45__14812 at 0x12161dd50>})

In [66]:
model.metabolites.f6p_B_c.reactions

frozenset({<Reaction TKT2h at 0x1174117d0>,
           <Reaction _6__45__PHOSPHOFRUCTO__45__2__45__KINASE__45__RXN at 0x1178683d0>,
           <Reaction _3__46__1__46__3__46__46__45__RXN at 0x121287fd0>,
           <Reaction MPAKI at 0x1212d4e90>,
           <Reaction TAh at 0x12130d210>,
           <Reaction PGI at 0x12130d4d0>,
           <Reaction BFBPh at 0x12130eed0>,
           <Reaction FRUCTOSE__45__6__45__PHOSPHATE__45__PHOSPHOKETOLASE__45__RXN at 0x1213822d0>,
           <Reaction L__45__GLN__45__FRUCT__45__6__45__P__45__AMINOTRANS__45__RXN__45__FRUCTOSE__45__6P__47__GLN__47____47__CPD__45__13469__47__GLT__46__31__46__ at 0x121382c90>,
           <Reaction ABFPTh at 0x12150de50>,
           <Reaction PFKh at 0x12154f890>,
           <Reaction RXN__45__14812 at 0x12161dd50>,
           <Reaction GLUCOSAMINE__45__6__45__P__45__DEAMIN__45__RXN at 0x121688e10>,
           <Reaction UFAGT at 0x1216bdd50>})

In [67]:
foo.swap_mets(model.metabolites.f6p_B_c,model.metabolites.f6p_c)

In [68]:
model.metabolites.f6p_B_c.remove_from_model()

In [69]:
model.metabolites.fdp_B_c.id = 'fdp_c'

In [70]:
model.reactions.G6PBDH.id = 'G6PDH'

In [71]:
model.reactions.RXN0__45__5260.name = 'Glycerol-3-phosphate dehydrogenase, mitochondrial'
model.reactions.RXN0__45__5260.id = 'G3PD5'
model.reactions.G3PD1irm.id = 'G3PD1'

## Fix up Ubiquinone Reactions

In [72]:
model.metabolites.UBIQUINONE__45__30_m.id = 'q6_m'

In [73]:
model.metabolites.q6_m.compartment = 'm'

In [74]:
model.metabolites.q6h2_c.compartment = 'c'

In [75]:
model.metabolites.q6h2_c.id = 'q6h2_m'

## Change Genenames to newest nomenclature

In [76]:
for gene in model.genes:
    gene.id = 'UMAG_' + gene.id[2:]

In [77]:
import re

In [78]:
for rxn in model.reactions:
    if rxn.gene_reaction_rule != '':
        rep_rule = re.sub('um','UMAG_',rxn.gene_name_reaction_rule)
        rxn.gene_reaction_rule = rep_rule

## Implement Erythritol Pathway

In [79]:
model.genes.UMAG_00109.reactions

frozenset({<Reaction DHAK at 0x1216d2d50>, <Reaction TRIOK at 0x1216e2350>})

In [80]:
model.reactions.DHAK

Id,DHAK
Name,GLYCERONE-KINASE-RXN
Stoichiometry,atp_c + dha_c <=> adp_c + dhap_c + h_c
GPR,UMAG_00109
Lower bound,-25.000000
Upper bound,25.000000


In [81]:
met = Metabolite(name='Erythrose',id='eryth_c',formula='C4H8O4',charge=0,compartment='c')

In [82]:
rxn = Reaction(id='E4PK',name='Erythrose-4-phosphate kinase',lower_bound=-25,upper_bound=25)

In [83]:
rxn.add_metabolites({model.metabolites.e4p_c:-1,
                     model.metabolites.adp_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.atp_c:1,
                    met:1},combine=False)

In [84]:
met2 = Metabolite(name='Erythritol',id='erythol_c',formula='C4H10O4',charge=0,compartment='c')

In [85]:
rxn2 = Reaction(id='ER',name='Erythrose reductase',lower_bound=0,upper_bound=25)

In [86]:
rxn2.add_metabolites({met:-1,
                     model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.nadp_c:1,
                    met2:1},combine=False)

In [87]:
model.add_reactions([rxn,rxn2])

In [88]:
model.reactions.E4PK.gene_reaction_rule = 'UMAG_00109'

In [89]:
model.add_demand(model.metabolites.erythol_c)

Id,DM_erythol_c
Name,Demand Erythritol
Stoichiometry,erythol_c -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


In [90]:
model.change_objective(model.reactions.DM_erythol_c)

In [91]:
model.solve().f

25.0

## MEL Biosynthesis Pathway

In [92]:
model.metabolites.gdpmann_c

Id,gdpmann_c
Name,GDP-alpha-D-mannose
Formula,C16H23N5O16P2


In [93]:
model.metabolites.erythol_c.formula

'C4H10O4'

#### Defining EMT1

In [94]:
met = Metabolite(name='Mannosylerythritol',id='manerythol_c',formula='C10H20O9',charge=0,compartment='c')

In [95]:
rxn = Reaction(id='EMT1',name='Glycosyltransferase',lower_bound=0,upper_bound=1000)

In [96]:
model.metabolites.gdp_c.charge

-3

In [97]:
rxn.add_metabolites({model.metabolites.erythol_c:-1,
                    model.metabolites.gdpmann_c:-1,
                    model.metabolites.gdp_c:1,
                    model.metabolites.h_c:1,
                    met:1},combine=False)

In [98]:
rxn.gene_reaction_rule = 'UMAG_10104'

In [99]:
rxn.check_mass_balance()

{}

#### Defining MAC1

In [100]:
met2 = Metabolite(name='hexoyl-2-mannosylerithritol',id='hex2manerythol_c',formula='C16H30O10',charge=0,compartment='c')

In [101]:
rxn2 = Reaction(id='MAC1',name='Mannosylerithritol C2 Acyltransferase',lower_bound=0,upper_bound=1000)

In [102]:
rxn2.gene_reaction_rule = 'UMAG_03116'

In [103]:
rxn2.add_metabolites({met:-1,
                    model.metabolites.hexACP_c:-1,
                    model.metabolites.ACP_c:1,
                    met2:1},combine=False)

In [104]:
model.metabolites.hexACP_c.formula = 'C17H31N2O8PRS'
model.metabolites.hexACP_c.charge = -1
model.metabolites.ACP_c.formula = 'C11H21N2O7PRS'
model.metabolites.ACP_c.charge = -1

In [105]:
rxn2.reaction

'hexACP_c + manerythol_c --> ACP_c + hex2manerythol_c'

In [106]:
rxn2.check_mass_balance()

{}

#### Defining MAC2

In [107]:
met3 = Metabolite(name='tetradecanoyl-3-hexoyl-2-mannosylerithritol',id='MEL_D_c',formula='C30H56O11',charge=0,compartment='c')

In [108]:
rxn3 = Reaction(id='MAC2',name='Mannosylerithritol C3 Acyltransferase',lower_bound=0,upper_bound=1000)

In [109]:
rxn3.gene_reaction_rule = 'UMAG_10636'

In [110]:
model.metabolites.myrsACP_c.formula = 'C25H47N2O8PRS'
model.metabolites.myrsACP_c.charge = -1

In [111]:
rxn3.add_metabolites({met2:-1,
                    model.metabolites.myrsACP_c:-1,
                    model.metabolites.ACP_c:1,
                    met3:1},combine=False)

In [112]:
rxn3.check_mass_balance()

{}

In [113]:
model.add_reactions([rxn,rxn2,rxn3])

In [114]:
model.repair()

#### Defining MAT1

In [115]:
met1 = Metabolite(name='MEL A',id='MEL_A_c',formula='C34H60O13',charge=0,compartment='c')
met2 = Metabolite(name='MEL B',id='MEL_B_c',formula='C32H58O12',charge=0,compartment='c')
met3 = Metabolite(name='MEL C',id='MEL_C_c',formula='C32H58O12',charge=0,compartment='c')

In [116]:
rxn1 = Reaction(id='MAT1a',name='MEL acetylase - MEL A',lower_bound=0,upper_bound=25)
rxn1.gene_reaction_rule = 'UMAG_03114'
rxn1.add_metabolites({model.metabolites.MEL_D_c:-1,
                    model.metabolites.accoa_c:-2,
                    model.metabolites.coa_c:2,
                    met1:1},combine=False)
rxn2 = Reaction(id='MAT1b',name='MEL acetylase  - MEL B',lower_bound=0,upper_bound=25)
rxn2.gene_reaction_rule = 'UMAG_03114'
rxn2.add_metabolites({model.metabolites.MEL_D_c:-1,
                    model.metabolites.accoa_c:-1,
                    model.metabolites.coa_c:1,
                    met2:1},combine=False)
rxn3 = Reaction(id='MAT1c',name='MEL acetylase  - MEL C',lower_bound=0,upper_bound=25)
rxn3.gene_reaction_rule = 'UMAG_03114'
rxn3.add_metabolites({model.metabolites.MEL_D_c:-1,
                    model.metabolites.accoa_c:-1,
                    model.metabolites.coa_c:1,
                    met3:1},combine=False)

In [117]:
model.add_reactions([rxn1,rxn2,rxn3])

#### Defining Mmf1

In [118]:
met1 = model.metabolites.MEL_A_c.copy()
met1.compartment = 'e'
met1.id = 'MEL_A_e'
met2 = model.metabolites.MEL_B_c.copy()
met2.compartment = 'e'
met2.id = 'MEL_B_e'
met3 = model.metabolites.MEL_C_c.copy()
met3.compartment = 'e'
met3.id = 'MEL_C_e'
met4 = model.metabolites.MEL_D_c.copy()
met4.compartment = 'e'
met4.id = 'MEL_D_e'

In [119]:
rxn1 = Reaction(id='MEL_Ate',name='MEL A Transporter',lower_bound=-1000,upper_bound=1000)
rxn1.add_metabolites({model.metabolites.MEL_A_c:-1,
                    met1:1},combine=False)
rxn2= Reaction(id='MEL_Bte',name='MEL B Transporter',lower_bound=-1000,upper_bound=1000)
rxn2.add_metabolites({model.metabolites.MEL_B_c:-1,
                    met2:1},combine=False)
rxn3 = Reaction(id='MEL_Cte',name='MEL C Transporter',lower_bound=-1000,upper_bound=1000)
rxn3.add_metabolites({model.metabolites.MEL_C_c:-1,
                    met3:1},combine=False)
rxn4 = Reaction(id='MEL_Dte',name='MEL D Transporter',lower_bound=-1000,upper_bound=1000)
rxn4.add_metabolites({model.metabolites.MEL_D_c:-1,
                    met4:1},combine=False)

In [120]:
rxn1.gene_reaction_rule = 'UMAG_03115'
rxn2.gene_reaction_rule = 'UMAG_03115'
rxn3.gene_reaction_rule = 'UMAG_03115'
rxn4.gene_reaction_rule = 'UMAG_03115'

In [121]:
model.add_reactions([rxn1,rxn2,rxn3,rxn4])

In [122]:
model.add_demand(model.metabolites.MEL_A_e,prefix='EX_')
model.add_demand(model.metabolites.MEL_B_e,prefix='EX_')
model.add_demand(model.metabolites.MEL_C_e,prefix='EX_')
model.add_demand(model.metabolites.MEL_D_e,prefix='EX_')

Id,EX_MEL_D_e
Name,Exchange tetradecanoyl-3-hexoyl-2-mannosylerithritol
Stoichiometry,MEL_D_e -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


In [123]:
for rxn in model.reactions:
    if not rxn.id.startswith('EX') or not rxn.id.startswith('DM'):
        if rxn.reversibility == True:
            rxn.change_bounds(lb=-1000,ub=1000)
        else:
            rxn.change_bounds(lb=0,ub=1000)

In [124]:
from cameo.util import TimeMachine

In [125]:
for met in ['manerythol_c','hex2manerythol_c','MEL_D_c','MEL_A_c','MEL_B_c','MEL_C_c','myrsACP_c','ddca_c','hdca_c']:
    with TimeMachine() as tm:
        model.add_demand(model.metabolites.get_by_id(met),time_machine=tm)
        model.change_objective('DM_' + met)
        print model.solve().f

553.571428571
300.0
110.526315789
96.3302752294
102.941176471
102.941176471
-1.27897692437e-13
225.935828877
162.5


# Ustilagic Acid Biosynthesis

#### Defining CYP1 and CYP2

In [126]:
model.metabolites.udpg_c

Id,udpg_c
Name,UDP-alpha-D-glucose
Formula,C15H22N2O17P2


In [127]:
model.metabolites.hdca_c

Id,hdca_c
Name,palmitate
Formula,C16H31O2


In [128]:
met1 = Metabolite(id='16hhdca_c',name='16-Hydroxy Hexadecanoate',charge=-1,compartment='c',formula='C16H31O3')

In [129]:
rxn1 = Reaction(id='HDCAMO',name='Hexadecanoate Monooxigenase - CYP1',lower_bound=0,upper_bound=1000)
rxn1.add_metabolites(metabolites={model.metabolites.hdca_c:-1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadp_c:1,
                    model.metabolites.h2o_c:1,
                    met1:1},combine=False)

In [130]:
met2 = Metabolite(id='1516dhhdca_c',name='15,16-dihydroxy Hexadecanoate',charge=-1,compartment='c',formula='C16H31O4')

In [131]:
rxn2 = Reaction(id='HHDCAMO',name='16-Hydroxy Hexadecanoate Monooxigenase - CYP2',lower_bound=0,upper_bound=1000)
rxn2.add_metabolites({met1:-1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadp_c:1,
                    model.metabolites.h2o_c:1,
                    met2:1},combine=False)

In [132]:
model.add_reactions([rxn1,rxn2])

#### Defining UGT1, (UAT1, FAS2, UHD1) and AHD1

In [133]:
met1 = Metabolite(id='cb1516dhdca_c',name='Cellobiosyl 15,16-dihydroxy Hexadecanoate',charge=-1,compartment='c',formula='C28H51O14')
met2 = Metabolite(id='UA_A_c',name='Hexoyl Ustilagic Acid - UA A',charge=-1, compartment='c',formula='C36H63O17')
met3 = Metabolite(id='UA_B_c',name='Hexoyl alpha-Hydroxy  Ustilagic Acid - UA B',charge=-1, compartment='c',formula='C36H63O18')
met4 = Metabolite(id='UA_C_c',name='Octyl Ustilagic Acid - UA C',charge=-1, compartment='c',formula='C38H67O17')
met5 = Metabolite(id='UA_D_c',name='Octyl alpha-Hydroxy Ustilagic Acid - UA D',charge=-1, compartment='c',formula='C38H67O18')

In [134]:
rxn1 = Reaction(id='UAGT',name='Ustilagic Acid glycosyl transferase - UGT1',lower_bound=0,upper_bound=1000)
rxn1.add_metabolites({model.metabolites.get_by_id('1516dhhdca_c'):-1,
                    model.metabolites.udpg_c:-2,
                    model.metabolites.udp_c:2,
                    model.metabolites.h_c:2,
                    met1:1},combine=False)

In [135]:
rxn2 = Reaction(id='UAHA',name='Ustilagic Acid hexyl acylase and acetylase - UAT1,FAS2,UHD1',lower_bound=0,upper_bound=1000)
rxn2.add_metabolites({met1:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.hexACP_c:-1,
                    model.metabolites.accoa_c:-1,
                    model.metabolites.ACP_c:1,
                    model.metabolites.h2o_c:1,
                    model.metabolites.coa_c:1,
                    model.metabolites.nadp_c:1,
                    met2:1},combine=False)

In [136]:
rxn3 = Reaction(id='UAAAO',name='UA A alpha oxidase',lower_bound=0,upper_bound=1000)
rxn3.add_metabolites({met2:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.h2o_c:1,
                    model.metabolites.nadp_c:1,
                    met3:1},combine=False)

In [137]:
model.metabolites.ocACP_c.formula = 'C19H35N2O8PRS'
model.metabolites.ocACP_c.charge = -1

In [138]:
rxn4 = Reaction(id='UAOA',name='Ustilagic Acid octyl acylase and acetylase - UAT1,FAS2,UHD1',lower_bound=0,upper_bound=1000)
rxn4.add_metabolites({met1:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.ocACP_c:-1,
                    model.metabolites.accoa_c:-1,
                    model.metabolites.ACP_c:1,
                    model.metabolites.h2o_c:1,
                    model.metabolites.coa_c:1,
                    model.metabolites.nadp_c:1,
                    met4:1},combine=False)

In [139]:
rxn5 = Reaction(id='UACAO',name='UA C alpha oxidase',lower_bound=0,upper_bound=1000)
rxn5.add_metabolites({met4:-1,
                    model.metabolites.o2_c:-1,
                    model.metabolites.nadph_c:-1,
                    model.metabolites.h_c:-1,
                    model.metabolites.h2o_c:1,
                    model.metabolites.nadp_c:1,
                    met5:1},combine=False)

In [140]:
model.add_reactions([rxn1,rxn2,rxn3,rxn4,rxn5])

#### Transport and Exchanges

In [141]:
met1 = model.metabolites.UA_A_c.copy()
met1.compartment = 'e'
met1.id = 'UA_A_e'
met2 = model.metabolites.UA_B_c.copy()
met2.compartment = 'e'
met2.id = 'UA_B_e'
met3 = model.metabolites.UA_C_c.copy()
met3.compartment = 'e'
met3.id = 'UA_C_e'
met4 = model.metabolites.UA_D_c.copy()
met4.compartment = 'e'
met4.id = 'UA_D_e'

In [142]:
rxn1 = Reaction(id='UA_Ate',name='UA A Transporter',lower_bound=-1000,upper_bound=1000)
rxn1.add_metabolites({model.metabolites.UA_A_c:-1,
                    met1:1},combine=False)
rxn2= Reaction(id='UA_Bte',name='UA B Transporter',lower_bound=-1000,upper_bound=1000)
rxn2.add_metabolites({model.metabolites.UA_B_c:-1,
                    met2:1},combine=False)
rxn3 = Reaction(id='UA_Cte',name='UA C Transporter',lower_bound=-1000,upper_bound=1000)
rxn3.add_metabolites({model.metabolites.UA_C_c:-1,
                    met3:1},combine=False)
rxn4 = Reaction(id='UA_Dte',name='UA D Transporter',lower_bound=-1000,upper_bound=1000)
rxn4.add_metabolites({model.metabolites.UA_D_c:-1,
                    met4:1},combine=False)

In [143]:
model.add_reactions([rxn1,rxn2,rxn3,rxn4])

In [144]:
model.add_demand(model.metabolites.UA_A_e,prefix='EX_')
model.add_demand(model.metabolites.UA_B_e,prefix='EX_')
model.add_demand(model.metabolites.UA_C_e,prefix='EX_')
model.add_demand(model.metabolites.UA_D_e,prefix='EX_')

Id,EX_UA_D_e
Name,Exchange Octyl alpha-Hydroxy Ustilagic Acid - UA D
Stoichiometry,UA_D_e -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


In [145]:
for met in ['UA_A_c','UA_B_c','UA_C_c','UA_D_c']:
    with TimeMachine() as tm:
        model.add_demand(model.metabolites.get_by_id(met),time_machine=tm)
        model.change_objective('DM_' + met)
        print model.solve().f

102.040816327
104.166666667
94.3396226415
97.4025974026


# Fatty Acid Biosynthesis Clean-up

In [146]:
clean_up_dict_rxn = {'ACONITATEHYDR__45__RXN':'ACONTb',
                    'ACOATAm':'ACOATA','ACCOAhi':'ACCOAC',
                    'MCOATAm':'MCOATA','KAS14_1':'KAS14',
                    '3OAR40_1':'3OAR40','_4__46__2__46__1__46__58__45__RXN':'3HAD40',
                    'RXN__45__9515':'EAR40y','3OAS60_1':'3OAS60',
                    'RXN__45__9518':'3OAR60','RXN__45__9520':'3HAD60',
                    'EAR60y_1':'EAR60y','RXN__45__9650':'3OAR80',
                    '3HAD80_1':'3HAD80','EAR80y_1':'EAR80y',
                    'RXN__45__9650':'3OAS80','RXN__45__9651':'3OAS100',
                    '3OAR100_1':'3OAR100','RXN__45__9655':'3HAD100',
                    'RXN__45__9530':'EAR100y','RXN__45__9652':'3OAS120',
                    'RXN__45__9532':'3OAR120','RXN__45__9533':'3HAD120',
                    'EAR120y_1':'EAR120y','3OAS140_1':'3OAS140',
                    '3OAR140_1':'3OAR140','3HAD140_1':'3HAD140',
                    'EAR140y_1':'EAR140y','RXN__45__9654':'3OAS160',
                    'RXN__45__9540':'3OAR160','_4__46__2__46__1__46__61__45__RXN':'3HAD160',
                    'RXN__45__9542':'EAR160y','RXN3O__45__1803':'3OAS180',
                    'RXN__45__9633':'3OAR180','RXN__45__9634':'3HAD180',
                    'RXN3O__45__5293':'EAR180y'}

In [147]:
clean_up_dict_met = {'THREO__45__DS__45__ISO__45__CITRATE_e':'icit_e',
                    'THREO__45__DS__45__ISO__45__CITRATE_c':'icit_c',
                    'THREO__45__DS__45__ISO__45__CITRATE_m':'icit_m',
                    '3hbutACP_c':'3haACP_c','Crotonyl__45__ACPs_c':'but2eACP_c',
                    'Trans__45__D2__45__decenoyl__45__ACPs_c':'tdec2eACP_c',
                    'R__45__3__45__hydroxydodecanoyl__45__ACPs_c':'3hddecACP_c',
                    '_3__45__oxo__45__stearoyl__45__ACPs_c':'3ooctdACP_c',
                    'R__45__3__45__hydroxystearoyl__45__ACPs_c':'3hoctaACP_c',
                    'Octadec__45__2__45__enoyl__45__ACPs_c':'toctd2eACP_c',
                    'Stearoyl__45__ACPs_c':'ocdcaACP_c'}

In [148]:
for key in clean_up_dict_rxn.keys():
    model.reactions.get_by_id(key).id = clean_up_dict_rxn[key]

In [149]:
model.metabolites.get_by_id('3haACP_c').reactions

frozenset({<Reaction _3__45__HYDROXYDECANOYL__45__ACP__45__DEHYDR__45__RXN at 0x121837550>,
           <Reaction _3__45__OXOACYL__45__ACP__45__REDUCT__45__RXN at 0x1218addd0>})

In [150]:
model.remove_reactions(model.metabolites.get_by_id('3haACP_c').reactions,remove_orphans=True)

In [151]:
for key in clean_up_dict_met.keys():
    model.metabolites.get_by_id(key).id = clean_up_dict_met[key]

In [152]:
model.remove_reactions(['RXN__45__9648','RXN__45__9653'])

In [153]:
model.reactions.get_by_id('3OAS80').add_metabolites({model.metabolites.malcoa_c:1,model.metabolites.malACP_c:-1,
                                                    model.metabolites.coa_c:-1,model.metabolites.ACP_c:1})

In [154]:
model.reactions.get_by_id('3OAS100').add_metabolites({model.metabolites.malcoa_c:1,model.metabolites.malACP_c:-1,
                                                     model.metabolites.coa_c:-1,model.metabolites.ACP_c:1})

In [155]:
model.reactions.get_by_id('3OAS120').add_metabolites({model.metabolites.malcoa_c:1,model.metabolites.malACP_c:-1,
                                                     model.metabolites.coa_c:-1,model.metabolites.ACP_c:1})

In [156]:
model.reactions.get_by_id('3OAS160').add_metabolites({model.metabolites.malcoa_c:1,model.metabolites.malACP_c:-1,
                                                     model.metabolites.coa_c:-1,model.metabolites.ACP_c:1})

In [157]:
model.reactions.get_by_id('3OAS180').add_metabolites({model.metabolites.malcoa_c:1,model.metabolites.malACP_c:-1,
                                                     model.metabolites.coa_c:-1,model.metabolites.ACP_c:1})

In [158]:
model.remove_reactions(['RXN__45__14972','RXN__45__14973','RXN__45__14954'])

In [261]:
model.reactions.ACCOAC.gene_reaction_rule = 'UMAG_04629 or UMAG_05131'

In [262]:
model.remove_reactions(['RXN__45__11497'])

In [159]:
import json

#### Fixing the map

In [160]:
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap9.json','r') as EscherMap:
    d = json.load(EscherMap)

for node in d[1]['nodes']:
    if 'bigg_id' in d[1]['nodes'][node]: 
        if d[1]['nodes'][node]['bigg_id'] in clean_up_dict_met.keys():
            d[1]['nodes'][node]['bigg_id'] = clean_up_dict_met[d[1]['nodes'][node]['bigg_id']]
            
for rxn in d[1]['reactions']:
    if 'bigg_id' in d[1]['reactions'][rxn]: 
        if d[1]['reactions'][rxn]['bigg_id'] in clean_up_dict_rxn.keys():
            d[1]['reactions'][rxn]['bigg_id'] = clean_up_dict_rxn[d[1]['reactions'][rxn]['bigg_id']]
            
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap9.json','w') as EscherMap:    
    json.dump(d,EscherMap)

# Some central carbon metabolism fixes

In [161]:
model.reactions.ACONTb.lower_bound = -1000

In [162]:
model.reactions.ACONTa.lower_bound = -1000

In [163]:
model.reactions.ACITL.gene_reaction_rule = 'UMAG_01005 or UMAG_05677'

In [164]:
model.remove_reactions(['CITL'])

In [165]:
model.reactions.TRANS_Isocitrate_Malate_Shuttle.gene_reaction_rule = model.reactions.TRANS_Cis_Aconitate_Malate_Shuttle.gene_reaction_rule

In [166]:
hco3 = Reaction(id='HCO3tm',name='Carbonate transport, mitochondria',lower_bound=-1000,upper_bound=1000)
hco3.add_metabolites({model.metabolites.hco3_c:-1,
                    model.metabolites.hco3_m:1},combine=False)

In [167]:
model.add_reaction(hco3)

### Glycolysis

In [168]:
model.add_demand(model.metabolites.pyr_e,prefix='EX_')

Id,EX_pyr_e
Name,Exchange pyruvate
Stoichiometry,pyr_e -->
GPR,
Lower bound,0.000000
Upper bound,1000.000000


In [169]:
for rxn in model.metabolites.pyr_e.reactions:
    if rxn.id.startswith('TRANS'):
        rxn.change_bounds(lb=-1000,ub=1000)

In [170]:
clean_gly_rxn = {'TRANS__45__RXNTFS__45__45__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__':'GLCt1',
                'PFKh':'PFK',
                'FBAm':'FBA',
                'GAPDHm':'GAPD',
                'PGKm':'PGK',
                'PGMm':'PGM',
                'ENOm':'ENO',
                'PYKm':'PYK'}

In [171]:
for key in clean_gly_rxn.keys():
    model.reactions.get_by_id(key).id = clean_gly_rxn[key]

In [172]:
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json','r') as EscherMap:
    d = json.load(EscherMap)

for rxn in d[1]['reactions']:
    if 'bigg_id' in d[1]['reactions'][rxn]: 
        if d[1]['reactions'][rxn]['bigg_id'] in clean_gly_rxn.keys():
            d[1]['reactions'][rxn]['bigg_id'] = clean_gly_rxn[d[1]['reactions'][rxn]['bigg_id']]
            
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json','w') as EscherMap:    
    json.dump(d,EscherMap)

In [173]:
constraint = model.add_ratio_constraint('PFK','G6PDH',0.23)

### Pentose-Phoshpate Pathway

In [174]:
# Moving the Glucose Oxidase to the Mitochondrial comparment because for all the genes Pedant indicated a presence in the 
# Inner Mitochondrial Membrane. And in the Cytosol, the reaction clashes with the PPP.

move_compartment(model,[model.reactions.GLUCOSE__45__OXIDASE__45__RXN],'m')

In [175]:
model.remove_reactions(['RXN__45__3341'])

In [176]:
model.reactions.GLUCONOLACT__45__RXN.gene_reaction_rule = 'UMAG_05603'

In [177]:
clean_ppp_rxn = {'GLUCOSE__45__OXIDASE__45__RXN':'GLUOXm',
                'PGLh':'PGL',
                'PGDHh':'PGDH',
                'GLUCONOLACT__45__RXN':'GLAC',
                'GNKh':'GNK',
                'RPEh':'RPE',
                'RIB5PISOM__45__RXN':'RPI',
                'PRPPSYN__45__RXN':'PRPPS',
                '_1TRANSKETO__45__RXN':'TKT1',
                'TAh':'TALA','TKT2h':'TKT2',
                'RIBOKIN__45__RXN':'RBK'}

In [178]:
clean_ppp_met = {'GLC__45__D__45__LACTONE_c':'g15lac_c',
                'RIBOSE__45__5P_c':'r5p_c'}

In [179]:
foo.swap_mets(model.metabolites.CPD__45__10330_c,model.metabolites.rib__D_c)
model.metabolites.CPD__45__10330_c.remove_from_model('subtractive')

In [180]:
for key in clean_ppp_rxn.keys():
        model.reactions.get_by_id(key).id = clean_ppp_rxn[key]

In [181]:
for key in clean_ppp_met.keys():
    model.metabolites.get_by_id(key).id = clean_ppp_met[key]

In [182]:
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json','r') as EscherMap:
    d = json.load(EscherMap)

for node in d[1]['nodes']:
    if 'bigg_id' in d[1]['nodes'][node]: 
        if d[1]['nodes'][node]['bigg_id'] in clean_ppp_met.keys():
            d[1]['nodes'][node]['bigg_id'] = clean_ppp_met[d[1]['nodes'][node]['bigg_id']]

for rxn in d[1]['reactions']:
    if 'bigg_id' in d[1]['reactions'][rxn]: 
        if d[1]['reactions'][rxn]['bigg_id'] in clean_ppp_rxn.keys():
            d[1]['reactions'][rxn]['bigg_id'] = clean_ppp_rxn[d[1]['reactions'][rxn]['bigg_id']]
            
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json','w') as EscherMap:    
    json.dump(d,EscherMap)

In [185]:
foo.find_common_reactions(model,['pep_c','pyr_c'])

--------------------------------------------------
Reaction's PYK3 reversibility is True
gtp_c + pyr_c <=> gdp_c + h_c + pep_c
--------------------------------------------------
Reaction's AGPOPm reversibility is True
dgtp_c + pyr_c <=> dgdp_c + h_c + pep_c
--------------------------------------------------
Reaction's DAPOP reversibility is False
datp_c + pyr_c --> dadp_c + h_c + pep_c
--------------------------------------------------
Reaction's PPS reversibility is False
atp_c + h2o_c + pyr_c --> amp_c + 2.0 h_c + pep_c + pi_c
--------------------------------------------------
Reaction's PYK reversibility is False
adp_c + h_c + pep_c --> atp_c + pyr_c


In [186]:
foo.invert_reaction(model.reactions.PYK3)

In [187]:
model.reactions.PYK3.lower_bound = 0

In [188]:
foo.invert_reaction(model.reactions.AGPOPm)

In [189]:
model.reactions.AGPOPm.lower_bound = 0

In [190]:
foo.invert_reaction(model.reactions.DAPOP)

# Mitochondrial Respiratiory Chain

#### Complex 1: NADH-Dehydrogenase:

In [191]:
# Unbalanced, and amount of transported protons didn't match KEGGs description here
# http://www.genome.jp/kegg-bin/show_pathway?uma00190

In [192]:
model.reactions.NADH__45__DEHYDROG__45__A__45__RXN

Id,NADH__45__DEHYDROG__45__A__45__RXN
Name,NADH-DEHYDROG-A-RXN
Stoichiometry,4.0 h_m + nadh_m + q6_m --> 4.0 h_p + nad_m + q6h2_m
GPR,UMAG_01311 and UMAG_12039 and UMAG_10607 and UMAG_11495 and UMAG_11731 and UMAG_10989 and UMAG_10847 and UMAG_02164 and UMAG_03669 and UMAG_10695 and UMAG_11162 and UMAG_00778 and UMAG_11170 and UMAG_00381 and UMAG_11038 and UMAG_05598 and UMAG_12001 and UMAG_02437 and UMAG_10820 and UMAG_11896 and UMAG_00512 and UMAG_11448 and UMAG_00634
Lower bound,0.000000
Upper bound,1000.000000


In [193]:
model.reactions.NADH__45__DEHYDROG__45__A__45__RXN.add_metabolites({model.metabolites.h_m:-2,
                                                                   model.metabolites.h_p:1})

#### Complex 2: Succinate Dehydrogenase

In [194]:
model.remove_reactions(['RXN__45__14971'])

In [195]:
model.reactions.SUCCINATE__45__DEHYDROGENASE__45__UBIQUINONE__45__RXN

Id,SUCCINATE__45__DEHYDROGENASE__45__UBIQUINONE__45__RXN
Name,SUCCINATE-DEHYDROGENASE-UBIQUINONE-RXN
Stoichiometry,q6_m + succ_m --> fum_m + q6h2_m
GPR,UMAG_04971 and UMAG_01172 and UMAG_00844 and UMAG_11744 and UMAG_03845
Lower bound,0.000000
Upper bound,1000.000000


#### Complex 3: Cytochrome bc1 Complex

In [196]:
# Unbalanced, and amount of transported protons didn't match KEGGs description here
# http://www.genome.jp/kegg-bin/show_pathway?uma00190

In [197]:
model.reactions.Ubiquinol_Cytochrome_C_Reductase_RXN

Id,Ubiquinol_Cytochrome_C_Reductase_RXN
Name,Aerobic Respiration Ubiquinol-Cytochrome-C-Reductase-RXN
Stoichiometry,2.0 Cytochromes__45__C__45__Oxidized_m + 2.0 h_m + q6h2_m <=> 2.0 focytC_m + 2.0 h_p + q6_m
GPR,UMAG_11097 and UMAG_10507 and UMAG_00940 and UMAG_11212 and UMAG_01478 and UMAG_04237 and UMAG_11288
Lower bound,-1000.000000
Upper bound,1000.000000


In [198]:
model.reactions.Ubiquinol_Cytochrome_C_Reductase_RXN.add_metabolites({model.metabolites.h_p:2})

#### Comple 4:  Cytochrome c Oxidase

In [199]:
# Unbalanced

In [200]:
model.reactions.Cytochrome_C_Oxidase_RXN

Id,Cytochrome_C_Oxidase_RXN
Name,Aerobic Respiration Cytochrome-C-Oxidase-RXN
Stoichiometry,focytC_m + 2.0 h_m + 0.5 o2_m --> Cytochromes__45__C__45__Oxidized_m + h2o_m + 2.0 h_p
GPR,UMAG_10048 and UMAG_12110 and UMAG_00912 and UMAG_11645 and UMAG_01150 and UMAG_11313 and UMAG_10296 and UMAG_03627 and UMAG_12241 and UMAG_11743 and UMAG_11153 and UMAG_05692 and UMAG_04802 and UMAG_12083 and UMAG_11005
Lower bound,0.000000
Upper bound,1000.000000


In [201]:
model.reactions.Cytochrome_C_Oxidase_RXN.add_metabolites({model.metabolites.h_m:-2})

#### AOX - Alternative Oxidase

In [202]:
RXN = Reaction(name='Alternative oxidase',id='AOXm')
RXN.gene_reaction_rule = 'UMAG_02774'

In [206]:
RXN.add_metabolites({model.metabolites.q6h2_m:-2,
                    model.metabolites.q6_m:2,
                    model.metabolites.o2_m:-1,
                    model.metabolites.h2o_m:2},combine=False)

In [207]:
RXN.check_mass_balance()

{}

In [211]:
model.add_reaction(RXN)

#### NDH-2

In [212]:
model.reactions.RXN0__45__5330.name = 'External alternative NADH-ubiquinone oxidoreductase, mitochondrial'

In [213]:
model.reactions.RXN0__45__5330

Id,RXN0__45__5330
Name,"External alternative NADH-ubiquinone oxidoreductase, mitochondrial"
Stoichiometry,nadh_m + q6_m --> h_m + nad_m + q6h2_m
GPR,UMAG_11038 or UMAG_12001 or UMAG_02437 or UMAG_10820 or UMAG_11896 or UMAG_00512 or UMAG_11448 or UMAG_00634
Lower bound,0.000000
Upper bound,1000.000000


In [214]:
model.reactions.RXN0__45__5330.add_metabolites({model.metabolites.h_c:-1,
                                               model.metabolites.nadh_m:0,
                                               model.metabolites.nad_m:0,
                                               model.metabolites.nadh_c:-1,
                                               model.metabolites.nad_c:1,
                                               model.metabolites.h_m:0},combine=False)

In [216]:
model.reactions.RXN0__45__5330

Id,RXN0__45__5330
Name,"External alternative NADH-ubiquinone oxidoreductase, mitochondrial"
Stoichiometry,h_c + nadh_c + q6_m --> nad_c + q6h2_m
GPR,UMAG_11038 or UMAG_12001 or UMAG_02437 or UMAG_10820 or UMAG_11896 or UMAG_00512 or UMAG_11448 or UMAG_00634
Lower bound,0.000000
Upper bound,1000.000000


#### ATPase

In [217]:
model.reactions.ATPasel

Id,ATPasel
Name,ATPSYN-RXN
Stoichiometry,adp_m + 3.0 h_p + pi_m --> atp_m + h2o_m + 3.0 h_m
GPR,UMAG_05503 and UMAG_10280 and UMAG_10180 and UMAG_02064 and UMAG_04716 and UMAG_11698 and UMAG_11802 and UMAG_11271 and UMAG_11576 and UMAG_10926 and UMAG_00508 and UMAG_10548 and UMAG_10213 and UMAG_04345 and UMAG_04167 and UMAG_11520 and UMAG_02782 and UMAG_01103
Lower bound,0.000000
Upper bound,1000.000000


In [218]:
model.reactions.ATPasel.check_mass_balance()

{'H': 1.0, 'charge': 1.0}

In [219]:
model.reactions.ATPasel.add_metabolites({model.metabolites.h_m:3,
                                        model.metabolites.h_p:-4},combine=False)

#### H-transporting ATPase

In [220]:
model.reactions.TRANS__45__RXN__45__249.check_mass_balance()

{}

In [221]:
model.reactions.TRANS__45__RXN__45__249.add_metabolites({model.metabolites.h_m:0,
                                        model.metabolites.h_p:1},combine=False)

In [222]:
model.reactions.TRANS__45__RXN__45__249.lower_bound = 0

In [223]:
model.reactions.TRANS__45__RXN__45__249

Id,TRANS__45__RXN__45__249
Name,TRANS-RXN-249
Stoichiometry,atp_m + h2o_m --> adp_m + h_p + pi_m
GPR,UMAG_05503 or UMAG_02064 or UMAG_04716 or UMAG_11698 or UMAG_11802 or UMAG_11576 or UMAG_10926 or UMAG_00508 or UMAG_10548 or UMAG_04345 or UMAG_04167 or UMAG_11520 or UMAG_02782 or UMAG_01103
Lower bound,0.000000
Upper bound,1000.000000


#### Glycerol phosphate dehydrogenase,

In [224]:
#implemented as
model.reactions.G3PD5

Id,G3PD5
Name,"Glycerol-3-phosphate dehydrogenase, mitochondrial"
Stoichiometry,glyc3p_c + q6_m --> dhap_c + q6h2_m
GPR,UMAG_01619
Lower bound,0.000000
Upper bound,1000.000000


In [225]:
clean_oxres_rxn = {'NADH__45__DEHYDROG__45__A__45__RXN':'NAD_umay',
                  'SUCCINATE__45__DEHYDROGENASE__45__UBIQUINONE__45__RXN':'SUCDH_q6_m',
                  'Ubiquinol_Cytochrome_C_Reductase_RXN':'CYOR_u6m_umay',
                  'Cytochrome_C_Oxidase_RXN':'CYOOm_umay',
                  'RXN0__45__5330':'NDH2m',
                  'TRANS__45__RXN__45__249':'ATPS'}

In [226]:
clean_oxres_met = {'Cytochromes__45__C__45__Oxidized_m':'ficytc_m',
                  'focytC_m':'focytc_m'}

In [227]:
for key in clean_oxres_rxn.keys():
        model.reactions.get_by_id(key).id = clean_oxres_rxn[key]

In [228]:
for key in clean_oxres_met.keys():
    model.metabolites.get_by_id(key).id = clean_oxres_met[key]

In [229]:
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json','r') as EscherMap:
    d = json.load(EscherMap)

for node in d[1]['nodes']:
    if 'bigg_id' in d[1]['nodes'][node]: 
        if d[1]['nodes'][node]['bigg_id'] in clean_oxres_met.keys():
            d[1]['nodes'][node]['bigg_id'] = clean_oxres_met[d[1]['nodes'][node]['bigg_id']]

for rxn in d[1]['reactions']:
    if 'bigg_id' in d[1]['reactions'][rxn]: 
        if d[1]['reactions'][rxn]['bigg_id'] in clean_oxres_rxn.keys():
            d[1]['reactions'][rxn]['bigg_id'] = clean_oxres_rxn[d[1]['reactions'][rxn]['bigg_id']]
            
with open('/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json','w') as EscherMap:    
    json.dump(d,EscherMap)

In [231]:
constraint2 = model.add_ratio_constraint('NDH2m','NAD_umay',0.5)

In [233]:
constraint3 = model.add_ratio_constraint('AOXm','CYOOm_umay',0.2)

In [242]:
model.metabolites.glc__D_e.reactions

frozenset({<Reaction EX_glc__D_e at 0x116fbd790>,
           <Reaction GLCt1 at 0x1212388d0>,
           <Reaction TRANS__45__RXNTFS__45__2__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__ at 0x121592150>,
           <Reaction TRANS__45__RXNTFS__45__32__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__ at 0x12164ae50>})

In [243]:
model.reactions.TRANS__45__RXNTFS__45__2__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__.lower_bound = 0
model.reactions.TRANS__45__RXNTFS__45__2__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__.upper_bound = 0

In [244]:
model.reactions.TRANS__45__RXNTFS__45__32__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__.lower_bound = 0
model.reactions.TRANS__45__RXNTFS__45__32__45__ALPHA__45__GLUCOSE__47____47__ALPHA__45__GLUCOSE__46__29__46__.upper_bound = 0

In [263]:
with TimeMachine() as tm:
    model.add_demand(model.metabolites.icit_m,time_machine=tm)
    model.change_objective(model.reactions.DM_icit_m)
    print model.solve().f

551.583837251


In [264]:
model.change_objective(model.reactions.BIOMASS_REACTION)

In [265]:
solution = model.solve()

In [266]:
solution.f

11.013093151059593

In [248]:
solution.fluxes['EX_pyr_e']

0.0

In [240]:
maps = escher.Builder(
                    model=model,
                    map_name = 'Ustilago maydis 521 genome-scale metabolic model',
                    map_json='/Users/clie/Desktop/Side Projects/Ustilago_maydis_Thiemo_Zambanini/UstiMap10.json',
                    reaction_data=solution.fluxes,
                    show_gene_reaction_rules=True,
                    use_3d_transform = True,
                    )

maps.set_reaction_scale([ { 'type': 'min', 'color': 'rgb(235,89,162)', 'size': 12 },
   { 'type': 'median', 'color': 'rgb(200,200,200)', 'size': 20 },
   { 'type': 'max', 'color': 'rgb(255,255,109)', 'size': 25 } ])
maps.display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


/Users/clie/.virtualenvs/modeling/lib/python2.7/site-packages/escher/plots.py:351: UserWarning:

map_json overrides map_name

127.0.0.1 - - [10/Oct/2016 00:48:28] "GET / HTTP/1.1" 200 -



stopping Server...


In [267]:
from cobra.io import write_sbml_model,save_json_model

In [268]:
write_sbml_model(model, 'iCL_Conversion3.xml', use_fbc_package=True)
save_json_model(model, 'iCL_Conversion3.json',pretty = True)

In [269]:
model2 = load_model('iCL_Conversion3.xml',sanitize=False)